In [ ]:
import mxnet as mx
from mxnet import autograd, gluon, nd
import time

In [ ]:
net = gluon.nn.Sequential()
net.add(
    gluon.nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
    gluon.nn.MaxPool2D(pool_size=2, strides=2),
    # TODO: fill in the rest of the LeNet network 
    
)

In [ ]:
batch_size = 256
transformer = gluon.data.vision.transforms.Compose([
    gluon.data.vision.transforms.ToTensor()
])

mnist_train = gluon.data.vision.FashionMNIST(train=True)
mnist_test = gluon.data.vision.FashionMNIST(train=False)

num_workers = 4

train_iter = gluon.data.DataLoader(mnist_train.transform_first(transformer),
                                   batch_size, shuffle=True,
                                   num_workers=num_workers)
test_iter = gluon.data.DataLoader(mnist_test.transform_first(transformer),
                                  batch_size, shuffle=False,
                                  num_workers=num_workers)

In [ ]:
ctx = mx.gpu(0)

In [ ]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train_metric = mx.metric.Accuracy()

In [ ]:
# TODO: tweak the learning rate between 0 and 1, run a couple of times and find the best setting.
lr, num_epochs = 0.1, 20
net.initialize(force_reinit=True, ctx=ctx, init=mx.init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

In [ ]:
def evaluate(data_iter, net, ctx):
    acc = nd.array([0], ctx=ctx)
    test_metric = mx.metric.Accuracy()
    test_metric.reset()
    for X, y in data_iter:
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
        test_metric.update(y, net(X))
    test_metric_name, test_metric_acc = test_metric.get()
    return test_metric_acc

In [ ]:
for epoch in range(num_epochs):
    train_l_sum, train_acc_sum, start = 0, 0, time.time()
    train_metric.reset()
    for X, y in train_iter:
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
        with autograd.record():
            y_hat = net(X)
            l = loss(y_hat, y)
        l.backward()
        trainer.step(batch_size)
        train_l_sum += l.mean().asscalar()
        train_metric.update(y, y_hat)
    train_metric_name, train_acc = train_metric.get()
    test_acc = evaluate(test_iter, net, ctx)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
          'time %.1f sec' % (epoch + 1, train_l_sum / len(train_iter),
                             train_acc_sum / len(train_iter),
                             test_acc, time.time() - start))